In [2]:
import json
import glob
import numpy as np
import pandas as pd
import os
from collections import defaultdict

In [3]:
from stats import get_dataframe

In [4]:
df = get_dataframe()

In [4]:
from scipy.special import betaln
def confidence_b_gr_a(pos_a, neg_a, pos_b, neg_b):
    alpha_a = pos_a + 1
    beta_a = neg_a + 1
    alpha_b = pos_b + 1
    beta_b = neg_b + 1
    
    total = 0.0
    for i in range(alpha_b):
        total += np.exp(
            betaln(alpha_a + i, beta_b + beta_a)
            - np.log(beta_b + i)
            - betaln(1 + i, beta_b)
            - betaln(alpha_a, beta_a)
        )
    return total

In [6]:
None

In [15]:
def compare_humans_and_bots(df, fields):
    human_stats = df[df.is_bot == False].groupby(fields + ['num_bots', 'win']).count().game
    bot_stats = df[df.is_bot == True].groupby(fields + ['num_bots', 'win']).count().game
    
    results = []
    for idx in human_stats.index:
        if not idx[-1]:
            continue
        num_bots = idx[-2]
        idx = idx[:-2]
        human_win = human_stats.loc[idx + (num_bots, True)]
        human_loss = human_stats.loc[idx + (num_bots, False)]
        bot_win = bot_stats.loc[idx + (num_bots + 1, True)]
        bot_loss = bot_stats.loc[idx + (num_bots + 1, False)]
        new_dict = {
            'num_other_bots': num_bots,
            'bot_winrate': bot_win / float(bot_win + bot_loss),
            'bot_n': bot_win + bot_loss,
            'human_winrate': human_win / float(human_win + human_loss),
            'human_n': human_win + human_loss,
            'bot_better_confidence': confidence_b_gr_a(human_win, human_loss, bot_win, bot_loss)
        }
        for k, v in zip(fields, idx):
            new_dict[k] = v
        results.append(new_dict)
    
    result = pd.DataFrame(results, columns=['num_other_bots'] + fields + ['bot_winrate', 'human_winrate', 'bot_better_confidence', 'bot_n', 'human_n'])
    return result.groupby(['num_other_bots'] + fields).mean()

compare_humans_and_bots(df, ['is_resistance'])

bot_winrate  human_winrate  \
num_other_bots is_resistance                               
0              False             0.696970       0.597015   
               True              0.500000       0.402985   
1              False             0.533742       0.564356   
               True              0.454902       0.383234   
2              False             0.675732       0.545098   
               True              0.356050       0.462366   
3              False             0.692423       0.628125   
               True              0.351218       0.324268   
4              False             0.656000       0.563953   
               True              0.344000       0.263087   

                              bot_better_confidence  bot_n  human_n  
num_other_bots is_resistance                                         
0              False                       0.864015     33      670  
               True                        0.873972     34     1005  
1              False                       0.315640    163      101  
               True                        0.926505    255      167  
2              False                       0.999743    478      255  
               True                        0.000339    719      372  
3              False                       0.988944   2006      320  
               True                        0.872604   3038      478  
4              False                       0.999968   3000      516  
               True                        0.999995   4500      745

In [16]:
compare_humans_and_bots(df[df.role == 'Assassin'], [])

,bot_winrate,human_winrate,bot_better_confidence,bot_n,human_n
num_other_bots,,,,,
0,0.619048,0.597015,0.556984,21,335
1,0.590361,0.577778,0.559193,83,45
2,0.680851,0.512000,0.999121,235,125
3,0.698397,0.625806,0.965484,998,155
4,0.656000,0.550201,0.999301,1500,249


In [17]:
compare_humans_and_bots(df[df.role == 'Merlin'], [])

,bot_winrate,human_winrate,bot_better_confidence,bot_n,human_n
num_other_bots,,,,,
0,0.500000,0.402985,0.742445,10,335
1,0.454545,0.392857,0.763237,88,56
2,0.382609,0.466102,0.067113,230,118
3,0.340704,0.282209,0.926813,995,163
4,0.344000,0.289683,0.953531,1500,252


In [18]:
compare_humans_and_bots(df, [])

,bot_winrate,human_winrate,bot_better_confidence,bot_n,human_n
num_other_bots,,,,,
0,0.597015,0.480597,0.968710,67,1675
1,0.485646,0.451493,0.808333,418,268
2,0.483709,0.496013,0.308750,1197,627
3,0.486915,0.446115,0.983943,5044,798
4,0.468800,0.386201,1.000000,7500,1261


# TrueSkill Stuff

In [48]:
import trueskill
trueskill.setup(draw_probability=0.0)

trueskill.TrueSkill(mu=25.000, sigma=8.333, beta=4.167, tau=0.083, draw_probability=0.0%)

In [74]:
indexed_by_game = df.set_index('game').sort_index()

fair_proportion = pd.concat([
    indexed_by_game[indexed_by_game.num_bots == 0][:60],
    indexed_by_game[indexed_by_game.num_bots == 1][:60],
    indexed_by_game[indexed_by_game.num_bots == 2][:60],
    indexed_by_game[indexed_by_game.num_bots == 3][:60],
    indexed_by_game[indexed_by_game.num_bots == 4][:60],
])
indexed_by_game = fair_proportion

In [61]:
def rate_game(bot_res, human_res, bot_spy, human_spy, game_id, df):
    game_data = df.loc[game_id]
    
    res_bots = game_data.res_bots[0]
    spy_bots = game_data.spy_bots[0]
    res_human = 3 - res_bots
    spy_human = 2 - spy_bots
    
    res_wins = game_data.resistance_win[0]
    
    return trueskill.rate(
        rating_groups=[(bot_res, human_res), (bot_spy, human_spy)],
        ranks=[int(not res_wins), int(res_wins)],
        weights=[(res_bots, res_human), (spy_bots, spy_human)]
    )

In [80]:
from scipy.special import erfc
bot_res = trueskill.Rating()
human_res = trueskill.Rating()
bot_spy = trueskill.Rating()
human_spy = trueskill.Rating()

for game_id in indexed_by_game.sample(frac=1).index.unique():
    (bot_res, human_res), (bot_spy, human_spy) = rate_game(bot_res, human_res, bot_spy, human_spy, game_id, indexed_by_game)

def better_confidence(sk1, sk2):
    return 0.5 * erfc( (sk2.mu - sk1.mu) / ((2 * (sk1.sigma ** 2 + sk2.sigma ** 2)) ** 0.5) )
    
pd.DataFrame([{
    'alliance': 'Resistance',
    'bot_mu': bot_res.mu,
    'bot_sigma': bot_res.sigma,
    'human_mu': human_res.mu,
    'human_sigma': human_res.sigma,
    'bot_better_confidence': better_confidence(bot_res, human_res)
},{
    'alliance': 'Spy',
    'bot_mu': bot_spy.mu,
    'bot_sigma': bot_spy.sigma,
    'human_mu': human_spy.mu,
    'human_sigma': human_spy.sigma,
    'bot_better_confidence': better_confidence(bot_spy, human_spy)
}], columns=['alliance', 'bot_mu', 'human_mu', 'bot_better_confidence', 'bot_sigma', 'human_sigma'])

,alliance,bot_mu,human_mu,bot_better_confidence,bot_sigma,human_sigma
0,Resistance,15.264451,11.542613,0.955932,1.751156,1.302593
1,Spy,25.578551,22.143179,0.882665,2.054007,2.034030
